# Load data from backend

In [ ]:
#@title Default title text
max_units =  0#@param {type:"integer"}
import sys
import requests
import json
import os.path
import math
import functools

headers = {'Cookie': '_orca_session='}
UNIT_URL = "https://orca.mova21.ch/units/"
UNIT_URL_JSON = "https://orca.mova21.ch/units.json"
ACTIVTY_URL = "https://orca.mova21.ch/activities/"
ACTIVTY_URL_JSON = "https://orca.mova21.ch/activities.json"
UNITS_FILE = "units.json"

# get activities from orca
r = requests.get(ACTIVTY_URL_JSON, headers=headers)
activities = r.json()

# read all units from backend
units = []


if os.path.exists(UNITS_FILE):   
  u = requests.get(UNIT_URL_JSON, headers=headers)
  u = u.json()        
  f = open(UNITS_FILE,)
  units = json.load(f)
else:
  for unit in u:    # to implement move get list of units firs 
    print("Reading unit " + str(unit.get("id")))
    r = requests.get(UNIT_URL + str(unit.get("id")) + ".json", headers=headers)
    units.append(r.json())
    if max_units > 0 and len(units) >= max_units:
      break

  with open(UNITS_FILE, 'w') as f:
    json.dump(units, f, ensure_ascii=False)

# Class Definitions

In [ ]:
# super class of activity
class Activity:
  def __init__(self, id, label, category, activity_executions):
    self.id = id
    self.label = label
    self.category = category
    self.activity_executions = activity_executions
    self.weighted_number_of_bookings = 0
    self.booked_units = []
    self.required_capacity_participants = 0
    self.required_capacity_leaders = 0
    self.required_capacity_units = 0
  
  def __repr__(self):
    return "id: " + str(self.id) + "\n label: " + self.label

  def addUnit(self, unit):
    self.booked_units.append(unit)

  def getNumberOfBookings(self):
    return len(self.booked_units)

  def getNumberOfParticipantBooked(self):
    return sum(unit.participant_count for unit in self.booked_units)

  def getNumberOfLeadersBooked(self):
    return sum(unit.leader_count for unit in self.booked_units)

  def getExecutionCapacity(self):
    return sum(activity_execution.amount_participants for activity_execution in self.activity_executions)

  def addRequiredCapacity(self, unit):
    self.required_capacity_participants += unit.participant_count 
    self.required_capacity_leaders += unit.leader_count
    self.required_capacity_units += 1

# representing activity object which units have booked, based on activity
# selected activity has further a priority
class SelectedActivity(Activity):
  def __init__(self, id, label, category, priority, activity_executions):
    super().__init__(id, label, category, activity_executions)
    self.priority = priority

  def __repr__(self):
    return str(self.id) + " - " + str(self.priority)

  # returns a score based on the priority used for the weighted summarization
  def getWeightedPriorityScore(self):
    # +1 to ensure none zero terms
    return (1/(self.priority + 1))**(1/3)

In [ ]:
class ActivityExecutions:
  def __init__(self, id, amount_participants, starts_at, ends_at, languages, spot, transport):
    self.id = id
    self.amount_participants = amount_participants
    self.starts_at = starts_at
    self.ends_at = ends_at
    self.languages = languages
    self.spot = spot
    self.transport = transport

In [ ]:
class Unit:
  def __init__(self, id, district, starts_at, ends_at, language, participant_count, leader_count, stufe, week, selected_activities):
    self.id = id
    self.district = district
    self.starts_at = starts_at
    self.ends_at = ends_at
    self.language = language
    self.participant_count = participant_count
    self.leader_count = leader_count
    self.stufe = stufe
    self.week = week
    self.selected_activities = selected_activities       # activities booked by the unit
    self.assigned_activities = []
    self.complies = False
    self.compliance_counts = {
      "mova_activity": 0, 
      "excursions": 0,
      "hiking": 0, 
      "village_global": 0, 
    }

  def __repr__(self):
    return "id" + str(self.id)

In [ ]:
# class which combines information and can be used to generate reports
class Report:
  def __init__(self):
    self.activities = []
    self.units = []
  
  def addActivity(self, activity):
    self.activities.append(activity)

  def addUnit(self, unit):
    self.units.append(unit)

    # add unit to report and to the corresponding activity object
    for sel_act in unit.selected_activities:
      act = self.getActivity(sel_act.id)
      
      if act:
        act.addUnit(unit)
        act.weighted_number_of_bookings += sel_act.getWeightedPriorityScore()
      else:
        raise Exception('Unit booked an none existing activity :(')

  # get activity object based on id
  def getActivity(self, id):
    return next((activity for activity in self.activities if activity.id == id), None)
  
  # generate a csv export of all bookings, each unit is listed for each of their bookings
  def getListOfBookings(self):
    logger.write("unit_id;" + "unit_stufe;" + "unit_language;" + "unit_#participant;" + "unit_#leaders;" + "unit_url;" + "act_id;" + "prio;" + "activity_url")
    for unit in self.units:
      for sel_act in unit.selected_activities:
        unit_url = UNIT_URL + str(unit.id)
        activity_url = ACTIVTY_URL + str(sel_act.id)
        logger.write(str(unit.id) + ";" + str(unit.stufe) + ";" + str(unit.language) + ";" + str(unit.participant_count) + ";" + str(unit.leader_count) + ";" + unit_url + ";" + str(sel_act.id) + ";" + str(sel_act.priority) + ";" +  activity_url)

  # generate csv export for activities with their corresponding bookings
  def getBookingCountOfActivity(self):
    sorted_activities = sorted(self.activities, key=lambda act: act.getNumberOfBookings(), reverse=True)

    logger.write("act_id;" + "#bookings;" + "#numberOfParticipantsAssigned;" + "#numberOfLeaders;" + "activity_url")
    for act in sorted_activities:
      activity_url = ACTIVTY_URL + str(act.id)
      logger.write(str(act.id) + ";" + str(act.getNumberOfBookings()) + ";" + str(act.getNumberOfParticipantBooked()) + ";" + str(act.getNumberOfLeadersBooked()) + ";" + activity_url)

  # generate csv export for activities with their corresponding bookings WEIGHTED by their priority
  def getBookingCountOfActivityWeighted(self):
    sorted_activities = sorted(self.activities, key=lambda act: act.weighted_number_of_bookings, reverse=True)

    logger.write("act_id;" + "#bookings;" + "activity_url")
    for act in sorted_activities:
      activity_url = ACTIVTY_URL + str(act.id)
      logger.write(str(act.id) + ";" + str(act.weighted_number_of_bookings) + ";" + activity_url)

  def getRequiredActivityCapacities(self):
    allocator = UnitActivityAllocator(self.units, self.activities)
    allocator.assign()
    logger.write("Aktivität;" + "Kategorie;" + "benötigte Plätze TN;"  + "benötigte Pläte inkl. Leiter;" + "benötigte Durchführungen;" +"Url")

    for act in self.activities:
      activity_url = ACTIVTY_URL + str(act.id)
      logger.write(str(act.id) + ";" + act.category + ";" + str(act.required_capacity_participants) + ";"  + str(act.required_capacity_leaders)  + ";"  + str(act.required_capacity_units) + ";" + activity_url)


  def getTheoreticalCapacities(self):
    theoretical_capacity_participants = 0
    theoretical_capacity_leaders = 0
    theoretical_capacity_units = 0
    multiplicator = 0
    
    for unit in self.units:
      if unit.stufe in ['pfadi', 'pio']:
        multiplicator = 5
      elif unit.stufe in ['pta', 'wolf']:
        multiplicator = 2
      else:
        multiplicator = 0

      theoretical_capacity_participants += unit.participant_count * multiplicator
      theoretical_capacity_leaders += unit.leader_count * multiplicator
      theoretical_capacity_units += 1 

    print(str(theoretical_capacity_participants))
    print(str(theoretical_capacity_leaders))
    print(str(theoretical_capacity_units))

  def getUnitCompliances(self):
    logger.write("Einheit;" + "Stufe;" + "# TN;" + "# Leiter;" + "min. # Aktivitäten;" + "# zugewiesene Aktivitäten;" + "Erfüllt;" + "# mova Aktivitäten;" + "# Ausflüge;" + "# Wanderungen;" + "# Village Global Workshops;" + " # Andere;")
    for unit in self.units:
      if unit.stufe in ['pfadi', 'pio']:
        multiplicator = 5
      elif unit.stufe in ['pta', 'wolf']:
        multiplicator = 2
      else:
        multiplicator = 0
      
      str_counts = [str(int) for int in unit.compliance_counts.values()]
      logger.write(str(unit.id) + ";" + unit.stufe + ";"  + str(unit.participant_count)  + ";"  + str(unit.leader_count) + ";" + str(multiplicator) + ";" + str(len(unit.assigned_activities)) + ";" + str(unit.complies) + ";" + ";".join(str_counts))


  def getListOfActivities(self):
    logger.write("act_id;" + "execution_capacity;" + "activity_url")

    for act in self.activities:
      activity_url = ACTIVTY_URL + str(act.id)
      logger.write(str(act.id) + ";" + str(act.getExecutionCapacity()) + ";" + activity_url)

In [ ]:
class Logger:
  def __init__(self, filename):
    self.console = sys.stdout
    self.file = open(filename, 'w')

  def write(self, message):
    self.console.write(message + '\n')
    self.file.write(message + '\n')

  def flush(self):
    self.console.flush()
    self.file.flush()

# Load scheduler with data

In [ ]:
report = Report()

# create activities
for activity in activities:
  act_executions = activity.get("activity_executions")

  executions = []
  for act_exe in act_executions:
    executions.append(ActivityExecutions(
                                         id = act_exe.get("id"), 
                                         amount_participants = act_exe.get("amount_participants"), 
                                         starts_at = act_exe.get("starts_at"), 
                                         ends_at = act_exe.get("ends_at"), 
                                         languages = act_exe.get("languages"), 
                                         spot = act_exe.get("spot"), 
                                         transport = act_exe.get("transport")))
    
  act = Activity(activity.get("id"), 
                 activity.get("label_in_database").get("de"),
                 activity.get("activity_category").get("code"),
                 activity_executions = executions)
  
  report.addActivity(act)

# create units
for unit in units:
  # if unit couldn't be found in backend, invalid unit ID queried
  if unit.get("id") is not None:
    selected_activities = []
    sel_activities = unit.get("unit_activities")

    # if unit did not make any booking
    if sel_activities: 
      for sel_act in sel_activities:
        # create SelectedActivity based on booked activity
        act = report.getActivity(sel_act.get("activity").get("id"))

        # if unit booked a none existing activity
        if act:
          selected_activities.append(SelectedActivity(act.id,
                                                      act.label,
                                                      act.category,
                                                      sel_act.get("priority"),
                                                      act.activity_executions))
        else:
          raise Exception('Unit ' + str(unit.get("id")) + ' booked an none existing activity :(')

        # reorder priority, so it is easier interpretable
        selected_activities = sorted(selected_activities, key=lambda selected_activity: selected_activity.priority)
    else:
      print('Unit ' + str(unit.get("id")) + ' did not make any bookings :(')

    # create unit
    un = Unit(
            id = unit.get("id"),
            district = unit.get("district"),
            starts_at = unit.get("starts_at"),
            ends_at = unit.get("ends_at"),
            language = unit.get("language"),
            participant_count = unit.get("participant_role_counts").get("participant") if unit.get("participant_role_counts") else 0,
            leader_count = unit.get("participant_role_counts").get("helper") if unit.get("participant_role_counts") else 0,
            stufe = unit.get("stufe"),
            week = unit.get("week"),
            selected_activities = selected_activities
    )

    report.addUnit(un)
  else:
    print('Unit ' + str(unit.get("id")) + ' does not exist :(')

# Allocator


In [ ]:
compliance_categorizers = {
    "mova_activity": lambda activity : activity.category == "activity", 
    "excursions": lambda activity : activity.category in ["culture", "excursion"],
    "hiking": lambda activity : activity.category == "hiking", 
    "village_global": lambda activity : activity.category == "village_global", 
    "other": lambda activity : activity.category not in ["activity", "hiking", "culture", "excursion", "village_global"]
}

class UnitActivityAllocator:
  def __init__(self, units, activities):
    self.activities = activities
    self.units = units

  def mova_activities_compliant(self, unit):
    # Pfadi/Pio: 3
    # Wolf: 1
    # PTA: 2
    count = unit.compliance_counts["mova_activity"]

    if unit.stufe in ["pfadi", "pio"]:
      if count >= 3: 
        return True
    elif unit.stufe == "pta":
      if count >= 2: 
        return True
    elif unit.stufe == "wolf":
      if count >= 2: 
        return True
    else: 
      return True;

    print("mova_activity is not yet satified")
    return False
  
  def hiking_compliant(self, unit):
    count = unit.compliance_counts["hiking"]

    if unit.stufe in ["pfadi", "pio"]:
      if count >= 1: 
        return True
    elif unit.stufe in ["wolf", "pta"]:
        return True
    else: 
      return True;

    return False
  
  def excursions_compliant(self, unit):
    # Pfadi/Pio: 2, wobei min 1 Wanderung und 2-Tageswanderung = 2
    # Wolf/PTA: 1
    excursion_count = unit.compliance_counts["excursions"]
    hiking_count = unit.compliance_counts["hiking"]
    combined_count = excursion_count + hiking_count

    if unit.stufe in ["pfadi", "pio"]:
      if combined_count >= 2: 
        return True
    elif unit.stufe in ["wolf", "pta"]:
      if combined_count >= 1: 
        return True
    else: 
      return True;

    return False

  def is_activity_assignable(self, unit, activity):
    category = self.activity_compliance_category(activity)
    if category:
      print("Activity " + activity.label + "(" + activity.category + ") belongs to the category " + category)

    if category == "mova_activity":  
      if self.mova_activities_compliant(unit): 
        return False
    if category == "hiking":  
      if self.hiking_compliant(unit): 
        return False
    elif category == "excursions":  
      if self.excursions_compliant(unit) or unit.compliance_counts["excursions"] >= 1: 
        return False
    elif category == "village_global": 
      if self.village_global_compliant(unit): 
        return False
    else: 
      return True;
    return True

  def activity_compliance_category(self, activity):
    for compliance_counter_name, compliance_counter in compliance_categorizers.items():
      if compliance_counter(activity):
        return compliance_counter_name

  def village_global_compliant(self, unit):
    # Alle: 1x mit x Workshops
    count = unit.compliance_counts["village_global"]
    needs = math.ceil(unit.participant_count / 12)

    if count >= needs: 
      return True

    print("village_global is not yet satified")
    return False

  def add_compliance_count(self, compliance_counts, activity):
    category = self.activity_compliance_category(activity)
    if category:
      compliance_counts[category] += 1
    return compliance_counts

  def recalculate_compliance_counts(self, unit):
    counts = {
      "mova_activity": 0, 
      "excursions": 0,
      "hiking": 0, 
      "village_global": 0, 
      "other": 0
    }

    for activity in unit.assigned_activities:
      counts = self.add_compliance_count(counts, activity)

    unit.compliance_counts = counts
    unit.complies = self.is_compliant(unit)
    return counts
  
  def is_compliant(self, unit):
    return self.mova_activities_compliant(unit) and \
           self.excursions_compliant(unit) and \
           self.village_global_compliant(unit) and \
           self.hiking_compliant(unit)

  def assign(self):
    for unit in self.units:
      self.assign_activities(unit)

  def get_activity(self, id):
    for activity in self.activities:
      if activity.id == id: 
        return activity

  def assign_activities(self, unit):
    print()
    print()
    print('Assigning activities to for unit ' + str(unit.id) + " (" + unit.stufe + ')')
    self.recalculate_compliance_counts(unit)
    for selected_activity in unit.selected_activities:
      print('Unit ' + str(unit.id) + ' wants activity ' + selected_activity.label + ' (' + str(selected_activity.id) + ')')
      if self.is_activity_assignable(unit, selected_activity):
        print('ok')
        unit.assigned_activities.append(selected_activity)
        activity = self.get_activity(selected_activity.id)
        if activity:
          activity.required_capacity_participants += unit.participant_count
          activity.required_capacity_leaders += unit.participant_count + unit.leader_count
          activity.required_capacity_units += 1
          unit.compliance_counts = self.add_compliance_count(unit.compliance_counts, activity)
        else:
          print("!!! Activity not found")
      else: 
        print("not ok:")
        print(unit.compliance_counts)

    self.recalculate_compliance_counts(unit)
    if self.is_compliant(unit):
      print('++++ Unit ' + str(unit.id) + ' is compliant and has ' + str(len(unit.assigned_activities)) + ' activities')
    else:
      print('---- Unit ' + str(unit.id) + ' is not compliant: ')
      print(unit.compliance_counts)

# Report Units

In [ ]:
logger = Logger(filename="activity_capacity.csv")
report.getListOfActivities()
logger.flush()

In [ ]:
logger = Logger(filename="bookings.csv")
report.getListOfBookings()
logger.flush()

In [ ]:
unit_count = 0
logger = Logger(filename="countOfActivities.csv")
report.getBookingCountOfActivity()
logger.flush()

In [ ]:
logger = Logger(filename="countOfActivitiesWeighted.csv")
report.getBookingCountOfActivityWeighted()
logger.flush()

In [ ]:
logger = Logger(filename="requiredCapacities.csv")
report.getRequiredActivityCapacities()
logger.flush()
logger = Logger(filename="unitCompliances.csv")
report.getUnitCompliances()
logger.flush()